# Player Data fetching from Sportradar and Supabase update

In [1]:
# Import necessary libraries
import os
import sys
from django.conf import settings
import pandas as pd

In [2]:
# Set up the Django environment
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hooperstats.settings")
import django
django.setup()

In [3]:
# Import utility functions
from api.sportradar_utils import get_sportradar_data
from api.supabase_utils import get_supabase_client

In [4]:
# Get Supabase client
supabase = get_supabase_client()

In [5]:
suns_team = supabase.table('teams').select('id').eq('name', 'Phoenix Suns').execute()
suns_id = suns_team.data[0]['id']
print(f"Phoenix Suns ID: {suns_id}")

Phoenix Suns ID: 583ecfa8-fb46-11e1-82cb-f4ce4684ea4c


In [6]:
# Step 2: Fetch Phoenix Suns players from Sportradar API
suns_data = get_sportradar_data(f'teams/{suns_id}/profile.json')
suns_data

{'id': '583ecfa8-fb46-11e1-82cb-f4ce4684ea4c',
 'name': 'Suns',
 'market': 'Phoenix',
 'alias': 'PHX',
 'founded': 1968,
 'sr_id': 'sr:team:3416',
 'owner': 'Mat Ishbia and Justin Ishbia',
 'general_manager': 'James Jones',
 'president': 'James Jones',
 'mascot': 'Go the Gorilla',
 'sponsor': 'PayPal',
 'championships_won': 0,
 'conference_titles': 3,
 'division_titles': 8,
 'retired_numbers': '5, 6, 7, 9, 13, 24, 31, 33, 34, 42, 44',
 'playoff_appearances': 33,
 'gleague_affiliate': 'Valley Suns',
 'reference': '1610612756',
 'venue': {'id': '1d2e59b2-2479-5f84-a6a7-e82f65dd8442',
  'name': 'Footprint Center',
  'capacity': 18422,
  'address': '201 E. Jefferson Street',
  'city': 'Phoenix',
  'state': 'AZ',
  'zip': '85004',
  'country': 'USA',
  'sr_id': 'sr:venue:6938',
  'location': {'lat': '33.445833', 'lng': '-112.071389'}},
 'league': {'id': '4353138d-4c22-4396-95d8-5f587d2df25c',
  'name': 'NBA',
  'alias': 'NBA'},
 'conference': {'id': '7fe7e212-de01-4f8f-a31d-b9f0a95731e3',
 

In [7]:
# Step 3: Extract players data
players_data = suns_data['players']

In [8]:
# Step 4: Transform players data into the format we need
players_to_insert = [
    {
        'id': player['id'],
        'name': player['full_name'],
        'team_id': suns_id
    }
    for player in players_data
]

In [10]:
# Step 5: Check for existing players and delete those not in the new set
existing_players = supabase.table('players').select('*').eq('team_id', suns_id).execute()
existing_ids = set(player['id'] for player in existing_players.data)
new_ids = set(player['id'] for player in players_to_insert)

# Players to delete (existing players not in the new set)
players_to_delete = existing_ids - new_ids
# Players to insert (new players not in the existing set)
players_to_insert = [player for player in players_to_insert if player['id'] in (new_ids - existing_ids)]
# Players to update (players in both sets)
players_to_update = [player for player in players_to_insert if player['id'] in (new_ids & existing_ids)]

print(f"Found {len(existing_ids)} existing players for the Suns.")
print(f"Fetched {len(new_ids)} players from the API.")
print(f"Will delete {len(players_to_delete)} players.")
print(f"Will insert {len(players_to_insert)} new players.")
print(f"Will update {len(players_to_update)} existing players.")

# Delete players not in the new set
if players_to_delete:
    delete_result = supabase.table('players').delete().in_('id', list(players_to_delete)).execute()
    print(f"Deleted {len(delete_result.data)} players no longer on the team.")

# Insert new players
if players_to_insert:
    insert_result = supabase.table('players').insert(players_to_insert).execute()
    print(f"Inserted {len(insert_result.data)} new players.")

# Update existing players
if players_to_update:
    for player in players_to_update:
        update_result = supabase.table('players').update(player).eq('id', player['id']).execute()
    print(f"Updated {len(players_to_update)} existing players.")

# Verify the final state
final_players = supabase.table('players').select('*').eq('team_id', suns_id).execute()
print(f"\nFinal count of Suns players in database: {len(final_players.data)}")

Found 0 existing players for the Suns.
Fetched 17 players from the API.
Will delete 0 players.
Will insert 17 new players.
Will update 0 existing players.
Inserted 17 new players.

Final count of Suns players in database: 17
